# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [26]:
cityData = pd.read_csv("C:\\Users\\12162\\Documents\\DataViz\\CWR-CLE-DATA-PT-01-2020-U-C\\06-Python-APIs\HomeworkAPI\\Solutions\\cities.csv")
cityData.head()

,Unnamed: 0,City,lon,temp,temp_max,humidity,cloudiness,wind_speed,country,lat
0,0,Uvira,29.15,73.00,73.00,48.0,15.0,0.47,CD,-3.41
1,1,Albany,-73.97,42.22,46.00,70.0,90.0,10.29,US,42.60
2,2,Nanortalik,-45.24,36.46,36.46,94.0,100.0,8.90,GL,60.14
3,3,City of San Pedro,121.02,81.77,82.40,69.0,20.0,5.82,PH,14.35
4,4,Kapaa,-159.32,70.88,73.40,83.0,40.0,13.87,US,22.08


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [27]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [30]:
# Heatmap of humidity
locations = cityData[["lat", "lon"]]
humidity = cityData["humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
# Narrow down cities that fit criteria and drop any results with null values
narrowedCity = city_data_df.loc[(city_data_df["temp_max"] < 80) & (cityData["temp_max"] > 70) \
                                    & (cityData["wind_speed"] < 10) \
                                    & (cityData["cloudiness"] == 0)].dropna()
narrowedCity

,Unnamed: 0,City,lon,temp,temp_max,humidity,cloudiness,wind_speed,country,lat
18,19,Lumding,93.17,71.17,71.17,30.0,0.0,2.21,IN,25.75
22,23,Port Elizabeth,25.57,71.60,71.60,78.0,0.0,6.93,ZA,-33.92
134,151,Bandar-e Lengeh,54.88,75.20,75.20,53.0,0.0,9.17,IR,26.56
208,238,Sawākin,37.33,75.96,75.96,60.0,0.0,7.16,SD,19.11
303,347,Makkah al Mukarramah,39.83,75.85,75.85,43.0,0.0,3.96,SA,21.43
323,375,Māngrol,70.12,75.56,75.56,76.0,0.0,8.68,IN,21.12
412,470,Sabha,14.43,71.35,71.35,18.0,0.0,6.78,LY,27.04
430,493,Bela,81.98,76.78,76.78,25.0,0.0,6.31,IN,25.93
432,496,Buraidah,43.98,73.40,73.40,33.0,0.0,5.82,SA,26.33
472,540,Srīvardhan,73.02,78.58,78.58,68.0,0.0,5.37,IN,18.03


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowedCity[["City", "country", "lat", "lon"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,country,lat,lon,Hotel Name
18,Lumding,IN,25.75,93.17,
22,Port Elizabeth,ZA,-33.92,25.57,
134,Bandar-e Lengeh,IR,26.56,54.88,
208,Sawākin,SD,19.11,37.33,
303,Makkah al Mukarramah,SA,21.43,39.83,
323,Māngrol,IN,21.12,70.12,
412,Sabha,LY,27.04,14.43,
430,Bela,IN,25.93,81.98,
432,Buraidah,SA,26.33,43.98,
472,Srīvardhan,IN,18.03,73.02,


In [43]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lon = row["lon"]
    
    params["location"] = f"{lat},{lon}"
    
    # Use the search term: "Hotel" and our lat/lon
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

,City,country,lat,lon,Hotel Name,Hotel
18,Lumding,IN,25.75,93.17,Hotel Nilamber,Hotel Nilamber
22,Port Elizabeth,ZA,-33.92,25.57,39 On Nile Guest House,39 On Nile Guest House
134,Bandar-e Lengeh,IR,26.56,54.88,Golestan Motel,Golestan Motel
208,Sawākin,SD,19.11,37.33,فندق الدويم,فندق الدويم
303,Makkah al Mukarramah,SA,21.43,39.83,Hotel Fairmont Makkah Clock Royal Tower,Hotel Fairmont Makkah Clock Royal Tower
323,Māngrol,IN,21.12,70.12,Hotel The Royal Honours,Hotel The Royal Honours
412,Sabha,LY,27.04,14.43,محطة ام الارانب -سبها مواصلات,محطة ام الارانب -سبها مواصلات
430,Bela,IN,25.93,81.98,Siddharth Hotel,Siddharth Hotel
432,Buraidah,SA,26.33,43.98,فندق راحة السلام,فندق راحة السلام
472,Srīvardhan,IN,18.03,73.02,Savli Resort/Hotel - Shrivardhan,Savli Resort/Hotel - Shrivardhan


In [50]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [51]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))